In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation ,Dropout, Flatten, Conv2D, MaxPooling2D
import pickle
from tensorflow import keras
import numpy as np
from tensorflow.keras import optimizers
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [3]:
x_train = pickle.load(open("x_train.data","rb"))
y_train = pickle.load(open("y_train.data","rb"))

In [4]:
import time
from tensorflow.keras.callbacks import TensorBoard
NAME = "Skin_Cancer_mar_18{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(NAME))

In [5]:
x_train.shape

(7511, 90, 120, 3)

In [6]:
model = Sequential()
model.add(Conv2D(32,kernel_size =(3,3),activation = 'relu', padding= 'same', input_shape = x_train.shape[1:]))
model.add(Conv2D(32,kernel_size =(3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(32,kernel_size =(3,3),activation = 'relu', padding= 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(64, (3,3),activation = 'relu', padding= 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Conv2D(128, (3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(128, (3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(128, (3,3),activation = 'relu', padding= 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Conv2D(256, (3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(256, (3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(256, (3,3),activation = 'relu', padding= 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Conv2D(32, (3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(32, (3,3),activation = 'relu', padding= 'same'))
model.add(Conv2D(32, (3,3),activation = 'relu', padding= 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(7,activation ='softmax'))
model.summary()
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 90, 120, 32)       896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 90, 120, 32)       9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 120, 32)       9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 45, 60, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 45, 60, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 45, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 30, 64)        0

In [7]:
from keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                        mode ="min", patience = 5,  
                                        restore_best_weights = True)

In [8]:
model.fit(x_train,y_train, batch_size=64,
                              epochs = 100,
          validation_split=0.1
                              , callbacks=[tensorboard,earlystopping])

Epoch 1/100
106/106 [==============================] - 101s 646ms/step - loss: 1.3646 - accuracy: 0.5846 - val_loss: 1.0833 - val_accuracy: 0.6662
Epoch 2/100
106/106 [==============================] - 61s 576ms/step - loss: 1.0031 - accuracy: 0.6735 - val_loss: 0.9406 - val_accuracy: 0.6662
Epoch 3/100
106/106 [==============================] - 110s 1s/step - loss: 0.9249 - accuracy: 0.6698 - val_loss: 0.9542 - val_accuracy: 0.6676
Epoch 4/100
106/106 [==============================] - 222s 2s/step - loss: 0.9157 - accuracy: 0.6658 - val_loss: 0.8878 - val_accuracy: 0.6702
Epoch 5/100
106/106 [==============================] - 257s 2s/step - loss: 0.8969 - accuracy: 0.6710 - val_loss: 0.9202 - val_accuracy: 0.6769
Epoch 6/100
106/106 [==============================] - 252s 2s/step - loss: 0.8818 - accuracy: 0.6721 - val_loss: 0.8962 - val_accuracy: 0.6862
Epoch 7/100
106/106 [==============================] - 252s 2s/step - loss: 0.8215 - accuracy: 0.6967 - val_loss: 0.8576 - val_accu

In [9]:
x_test = pickle.load(open("x_test.data","rb"))
y_test = pickle.load(open("y_test.data","rb"))

In [10]:
ct = model.predict(x_test)

In [11]:
a=0
b=0
for i,j in zip(ct, y_test):
    print(np.argmax(i),np.argmax(j),j)
    if np.argmax(i)==np.argmax(j):
        a+=1
    else:
        b+=1
        
print(a,b,a+b)

2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 1 [0. 1. 0. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4

4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 0 [1. 0. 0. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 4 [0. 0. 0. 0. 1. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4

4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6 6 [0. 0. 0. 0. 0. 0. 1.]
4

4 2 [0. 0. 1. 0. 0. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 3 [0. 0. 0. 1. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 0 [1. 0. 0. 0. 0. 0. 0.]
2 1 [0. 1. 0. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4

4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 0 [1. 0. 0. 0. 0. 0. 0.]
6 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2

In [12]:
from tensorflow import lite
import tensorflow as tf
kearas_file = "skincan74_ep19.h5"
tf.keras.models.save_model(model,kearas_file)
converter = lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("skincan74_ep19.tflite","wb").write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\Ak190\AppData\Local\Temp\tmpq29_p0e4\assets


8060396

In [13]:
model.fit(x_train,y_train, batch_size=64,
                              epochs = 10,
          validation_split=0.1
                              , callbacks=[tensorboard])

Epoch 1/10
106/106 [==============================] - 50s 472ms/step - loss: 0.7361 - accuracy: 0.7291 - val_loss: 0.7632 - val_accuracy: 0.7154
Epoch 2/10
106/106 [==============================] - 50s 476ms/step - loss: 0.7347 - accuracy: 0.7273 - val_loss: 0.7387 - val_accuracy: 0.7487
Epoch 3/10
106/106 [==============================] - 79s 747ms/step - loss: 0.7250 - accuracy: 0.7329 - val_loss: 0.7711 - val_accuracy: 0.7287
Epoch 4/10
106/106 [==============================] - 105s 993ms/step - loss: 0.7078 - accuracy: 0.7393 - val_loss: 0.7577 - val_accuracy: 0.7247
Epoch 5/10
106/106 [==============================] - 125s 1s/step - loss: 0.7056 - accuracy: 0.7406 - val_loss: 0.7632 - val_accuracy: 0.7181
Epoch 6/10
106/106 [==============================] - 129s 1s/step - loss: 0.6969 - accuracy: 0.7443 - val_loss: 0.7266 - val_accuracy: 0.7487
Epoch 7/10
106/106 [==============================] - 129s 1s/step - loss: 0.6830 - accuracy: 0.7446 - val_loss: 0.7566 - val_accurac

In [14]:
ct = model.predict(x_test)
a=0
b=0
for i,j in zip(ct, y_test):
    print(np.argmax(i),np.argmax(j),j)
    if np.argmax(i)==np.argmax(j):
        a+=1
    else:
        b+=1
        
print(a,b,a+b," acc ",a/(a+b)*(100),"&")

4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
5 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 1 [0. 1. 0. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
1 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4

4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
0 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4

1 1 [0. 1. 0. 0. 0. 0. 0.]
6 1 [0. 1. 0. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
2 3 [0. 0. 0. 1. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4

4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
0 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4

4 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
0 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
5 1 [0. 1. 0. 0. 0. 0. 0.]
2 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6 3 [0. 0. 0. 1. 0. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 1 [0. 1. 0. 0. 0. 0. 0.]
4

In [15]:
model.fit(x_train,y_train, batch_size=64,
                              epochs = 10,
          validation_split=0.1
                              , callbacks=[tensorboard])

Epoch 1/10
106/106 [==============================] - 50s 467ms/step - loss: 0.6267 - accuracy: 0.7708 - val_loss: 0.7437 - val_accuracy: 0.7527
Epoch 2/10
106/106 [==============================] - 49s 466ms/step - loss: 0.6207 - accuracy: 0.7738 - val_loss: 0.7577 - val_accuracy: 0.7154
Epoch 3/10
106/106 [==============================] - 70s 662ms/step - loss: 0.6112 - accuracy: 0.7784 - val_loss: 0.7435 - val_accuracy: 0.7394
Epoch 4/10
106/106 [==============================] - 97s 914ms/step - loss: 0.5831 - accuracy: 0.7834 - val_loss: 0.7532 - val_accuracy: 0.7500
Epoch 5/10
106/106 [==============================] - 118s 1s/step - loss: 0.5702 - accuracy: 0.7887 - val_loss: 0.7403 - val_accuracy: 0.7420
Epoch 6/10
106/106 [==============================] - 130s 1s/step - loss: 0.5397 - accuracy: 0.8012 - val_loss: 0.8122 - val_accuracy: 0.7128
Epoch 7/10
106/106 [==============================] - 151s 1s/step - loss: 0.5251 - accuracy: 0.8112 - val_loss: 0.7662 - val_accuracy

In [16]:
ct = model.predict(x_test)
a=0
b=0
for i,j in zip(ct, y_test):
    print(np.argmax(i),np.argmax(j),j)
    if np.argmax(i)==np.argmax(j):
        a+=1
    else:
        b+=1
        
print(a,b,a+b," acc ",a/(a+b)*(100),"%")

2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
5 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 1 [0. 1. 0. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4

4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 0 [1. 0. 0. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 0 [1. 0. 0. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4

4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6 6 [0. 0. 0. 0. 0. 0. 1.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
0 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 3 [0. 0. 0. 1. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6

4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
1 0 [1. 0. 0. 0. 0. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 1 [0. 1. 0. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4

4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 0 [1. 0. 0. 0. 0. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 3 [0. 0. 0. 1. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6 6 [0. 0. 0. 0. 0. 0. 1.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 0 [1. 0. 0. 0. 0. 0. 0.]
0 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
5 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4

In [17]:
model.fit(x_train,y_train, batch_size=64,
                              epochs = 10,
          validation_split=0.1
                              , callbacks=[tensorboard])

Epoch 1/10
106/106 [==============================] - 64s 603ms/step - loss: 0.4402 - accuracy: 0.8457 - val_loss: 0.8563 - val_accuracy: 0.7420
Epoch 2/10
106/106 [==============================] - 182s 2s/step - loss: 0.4025 - accuracy: 0.8578 - val_loss: 0.8335 - val_accuracy: 0.7460
Epoch 3/10
106/106 [==============================] - 252s 2s/step - loss: 0.3530 - accuracy: 0.8738 - val_loss: 0.9012 - val_accuracy: 0.7407
Epoch 4/10
106/106 [==============================] - 252s 2s/step - loss: 0.3274 - accuracy: 0.8847 - val_loss: 1.0573 - val_accuracy: 0.7301
Epoch 5/10
106/106 [==============================] - 252s 2s/step - loss: 0.3465 - accuracy: 0.8812 - val_loss: 1.0129 - val_accuracy: 0.7380
Epoch 6/10
106/106 [==============================] - 252s 2s/step - loss: 0.3098 - accuracy: 0.8899 - val_loss: 1.0677 - val_accuracy: 0.7207
Epoch 7/10
106/106 [==============================] - 252s 2s/step - loss: 0.3114 - accuracy: 0.8929 - val_loss: 1.0511 - val_accuracy: 0.73

In [18]:
ct = model.predict(x_test)
a=0
b=0
for i,j in zip(ct, y_test):
    print(np.argmax(i),np.argmax(j),j)
    if np.argmax(i)==np.argmax(j):
        a+=1
    else:
        b+=1
        
print(a,b,a+b," acc ",a/(a+b)*(100),"%")

2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
5 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
2 1 [0. 1. 0. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
0

4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
6 3 [0. 0. 0. 1. 0. 0. 0.]
0 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
1 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
3 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 2 [0. 0. 1. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4

4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
0 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
0 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 3 [0. 0. 0. 1. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 0 [1. 0. 0. 0. 0. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
5

4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
0 2 [0. 0. 1. 0. 0. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
1 0 [1. 0. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
1 5 [0. 0. 0. 0. 0. 1. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
0 1 [0. 1. 0. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
2 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
3 6 [0. 0. 0. 0. 0. 0. 1.]
5 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4

4 4 [0. 0. 0. 0. 1. 0. 0.]
3 3 [0. 0. 0. 1. 0. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
1 1 [0. 1. 0. 0. 0. 0. 0.]
5 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 1 [0. 1. 0. 0. 0. 0. 0.]
5 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 2 [0. 0. 1. 0. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
3 3 [0. 0. 0. 1. 0. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4 5 [0. 0. 0. 0. 0. 1. 0.]
2 4 [0. 0. 0. 0. 1. 0. 0.]
4 4 [0. 0. 0. 0. 1. 0. 0.]
4